<a href="https://colab.research.google.com/github/Manos-Mak120/-/blob/Articles-Scrapper/NEWSBOMB_TRUMP_ARTICLES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')
def get_article_elements_from_soup(soup, article_url):
    # title
    try:
        title = soup.find("h1").text.strip()
    except:
        title = ""

    # datetime
    try:
        datetime_str = soup.find("time")["datetime"]
        date_str = datetime_str.split("T")[0]
        time_str = datetime_str.split("T")[1]
    except:
        date_str = ""
        time_str = ""
    # section
    try:
        section_div = soup.find("div", class_="main-category-block") or soup.find("div", class_="main-category")
        section = section_div.text.strip() if section_div else ""
    except:
        section = ""

    # author
    try:
      author_tag = soup.find("a", class_="author-name") or soup.find("span", class_="author-name")
      author = author_tag.text.strip() if author_tag else ""
    except:
        author = ""
    # excerpt
    try:
        excerpt_div = soup.find("div", class_="p.main-intro") or soup.find(itemprop="description") or soup.find("div", class_="nx-excerpt")
        excerpt = excerpt_div.text.strip() if excerpt_div else ""
    except:
        excerpt = ""

    # text body
    try:
      content_div = soup.find("div", class_="main-text story-fulltext")
      elements = content_div.find_all(["p", "h2", "h3", "ul", "ol", "li"]) if content_div else []
      text_body = " ".join(el.get_text(strip=True) for el in elements if el.get_text(strip=True))
    except:
        text_body = ""

    # feature image
    try:
        img_block = soup.find("div", class_="article-image") or soup.find("figure")
        feat_img_link = img_block.find("img")["src"] if img_block and img_block.find("img") else ""
    except:
        feat_img_link = ""

    # image caption
    try:
        caption_div = soup.find("div", class_="image-caption") or soup.find("figcaption")
        image_caption = caption_div.text.strip() if caption_div else ""
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_cation": image_caption
    }

    return articleD

In [ ]:
def scrape_newsbomb_articles_pages(teasers_df):
  all_articles_dataL = []
  base_url = "https://www.newsbomb.gr"
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    article_url_relative = row['article_url']
    article_url = base_url + article_url_relative
    soup = get_webpage_soup(article_url)
    if soup:
     articleD = get_article_elements_from_soup(soup, article_url)
     all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trump_teasers_df = pd.read_csv("/content/drive/MyDrive/Εργασία Python/scrappers/trump_df_2024_25.csv")
trump_teasers_df.head(100)

,title,article_url,date,time,section,excerpt
0,Ο Ζελένσκι αποκάλυψε τι συζήτησε με τον Τραμπ ...,/kosmos/story/1646524/o-zelenski-apokalypse-ti...,03.05.2025,14:16,ΚΟΣΜΟΣ,Οι δύο ηγέτες είχαν μακρά συνομιλία στο Βατικανό.
1,Τραμπ: Επιλέγει για επόμενο Πάπα τον εαυτό του...,/kosmos/story/1646476/tramp-epilegei-gia-epome...,03.05.2025,11:43,ΚΟΣΜΟΣ,"Η εικόνα, κατά τα φαινόμενα φτιαγμένη από δημι..."
2,ΗΠΑ: Ο πρώτος αντίκτυπος από τους δασμούς στην...,/kosmos/story/1646448/ipa-o-protos-antiktypos-...,03.05.2025,08:25,ΚΟΣΜΟΣ,Ο μεγαλύτερος αντίκτυπος σημειώθηκε στις ίδιες...
3,ΗΠΑ: Έρχονται μεγάλες περικοπές στη CIA και άλ...,/kosmos/story/1646418/ipa-erxontai-megales-per...,03.05.2025,00:45,ΚΟΣΜΟΣ,Η CIA σκοπεύει να περικόψει 1.200 θέσεις εργασ...
4,Αμερικανορωσικός «πόλεμος» για την Ημέρα της Ν...,/kosmos/story/1646400/amerikanorosikos-polemos...,02.05.2025,22:38,ΚΟΣΜΟΣ,Τι ανέφερε η εκπρόσωπος του ρωσικού Υπουργείου...
...,...,...,...,...,...,...
95,Νέα άνοδος στη Wall Street μετά τις ελπίδες γι...,/oikonomia/story/1644136/nea-anodos-sti-wall-s...,24.04.2025,03:14,ΟΙΚΟΝΟΜΙΑ,Νέα ισχυρή άνοδο κατέγραψαν την Τετάρτη του Πά...
96,Τραμπ για πόλεμο στην Ουκρανία: «Νομίζω έχουμε...,/kosmos/story/1644140/tramp-gia-polemo-stin-ou...,24.04.2025,01:58,ΚΟΣΜΟΣ,Αισιόδοξος ο πρόεδρος των ΗΠΑ για τον τερματισ...
97,Αλέξης Τσίπρας: Τέσσερα «stop» για τους εχθρού...,/politikh/story/1644123/aleksis-tsipras-tesser...,23.04.2025,23:15,ΠΟΛΙΤΙΚΗ,"Όσα είπε ο Αλέξης Τσίπρας για την Ουκρανία, τη..."
98,Τραμπ: Είμαστε κοντά σε συμφωνία για την Ουκρα...,/kosmos/story/1644084/tramp-eimaste-konta-se-s...,23.04.2025,19:22,ΚΟΣΜΟΣ,Διπλωματικός πυρετός για την πολυπόθητη ειρήνη...


In [ ]:
trump_teasers_df_test = trump_teasers_df.head(10)
trump_teasers_df_test

,title,article_url,date,time,section,excerpt
0,Ο Ζελένσκι αποκάλυψε τι συζήτησε με τον Τραμπ ...,/kosmos/story/1646524/o-zelenski-apokalypse-ti...,03.05.2025,14:16,ΚΟΣΜΟΣ,Οι δύο ηγέτες είχαν μακρά συνομιλία στο Βατικανό.
1,Τραμπ: Επιλέγει για επόμενο Πάπα τον εαυτό του...,/kosmos/story/1646476/tramp-epilegei-gia-epome...,03.05.2025,11:43,ΚΟΣΜΟΣ,"Η εικόνα, κατά τα φαινόμενα φτιαγμένη από δημι..."
2,ΗΠΑ: Ο πρώτος αντίκτυπος από τους δασμούς στην...,/kosmos/story/1646448/ipa-o-protos-antiktypos-...,03.05.2025,08:25,ΚΟΣΜΟΣ,Ο μεγαλύτερος αντίκτυπος σημειώθηκε στις ίδιες...
3,ΗΠΑ: Έρχονται μεγάλες περικοπές στη CIA και άλ...,/kosmos/story/1646418/ipa-erxontai-megales-per...,03.05.2025,00:45,ΚΟΣΜΟΣ,Η CIA σκοπεύει να περικόψει 1.200 θέσεις εργασ...
4,Αμερικανορωσικός «πόλεμος» για την Ημέρα της Ν...,/kosmos/story/1646400/amerikanorosikos-polemos...,02.05.2025,22:38,ΚΟΣΜΟΣ,Τι ανέφερε η εκπρόσωπος του ρωσικού Υπουργείου...
5,Ηνωμένες Πολιτείες: Με προτεραιότητα στην Άμυν...,/kosmos/story/1646367/inomenes-politeies-me-pr...,02.05.2025,20:15,ΚΟΣΜΟΣ,Ο Λευκός Οίκος ανέφερε πως το προσχέδιο του αμ...
6,Bloomberg: «Plan B» του Τραμπ για το τέλος του...,/kosmos/story/1646362/bloomberg-plan-b-tou-tra...,02.05.2025,19:55,ΚΟΣΜΟΣ,Οι διπλωματικές προσπάθειες συνεχίζονται προς ...
7,Τραμπ: «Ημέρα Νίκης» ονομάζονται η 11η Νοεμβρί...,/kosmos/story/1646358/tramp-imera-nikis-onomaz...,02.05.2025,19:32,ΚΟΣΜΟΣ,Οι ημερομηνίες - ορόσημο των δύο Παγκοσμίων Πο...
8,Μήνυμα Τραμπ προς τη Fed: «Να μειώσει τώρα τα ...,/kosmos/story/1646324/minyma-tramp-pros-ti-fed...,02.05.2025,17:37,ΚΟΣΜΟΣ,«Η οικονομία μας βρίσκεται σε μεταβατικό στάδι...
9,Ο Τραμπ θα αφαιρέσει από το Harvard το αφορολό...,/kosmos/story/1646290/o-tramp-tha-afairesei-ap...,02.05.2025,15:33,ΚΟΣΜΟΣ,Από την ανάληψη των καθηκόντων του τον Ιανουάρ...


In [ ]:
articles_df = scrape_newsbomb_articles_pages(trump_teasers_df)
articles_df

Processing rows: 100%|██████████| 2457/2457 [1:44:31<00:00,  2.55s/it]



Scraping completed. Fetshed 2457 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/kosmos/story/1646524/o...,Ο Ζελένσκι αποκάλυψε τι συζήτησε με τον Τραμπ ...,2025-05-03,14:16:27+03:00,ΚΟΣΜΟΣ,Newsbomb,Οι δύο ηγέτες είχαν μακρά συνομιλία στο Βατικανό.,Ο Ουκρανός πρόεδροςΒολοντίμιρ Ζελένσκιδήλωσε ό...,https://img.bbmd.gr/img/1260/max/82/2025/04/30...,Copyright 2025 The Associated Press. All right...
1,https://www.newsbomb.gr/kosmos/story/1646476/t...,Τραμπ: Επιλέγει για επόμενο Πάπα τον εαυτό του...,2025-05-03,11:43:15+03:00,ΚΟΣΜΟΣ,Newsbomb,"Η εικόνα, κατά τα φαινόμενα φτιαγμένη από δημι...",ΟΝτόναλντ Τραμπαναδημοσίευσε χθεςΠαρασκευήμέσω...,https://img.bbmd.gr/img/1260/max/82/2025/05/03...,
2,https://www.newsbomb.gr/kosmos/story/1646448/i...,ΗΠΑ: Ο πρώτος αντίκτυπος από τους δασμούς στην...,2025-05-03,08:25:35+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο μεγαλύτερος αντίκτυπος σημειώθηκε στις ίδιες...,"Oι δασμοί που επέβαλε ο Αμερικανός πρόεδρος,Ντ...",https://img.bbmd.gr/img/1260/max/82/2025/04/27...,AP.
3,https://www.newsbomb.gr/kosmos/story/1646418/i...,ΗΠΑ: Έρχονται μεγάλες περικοπές στη CIA και άλ...,2025-05-03,00:45:26+03:00,ΚΟΣΜΟΣ,Newsbomb,Η CIA σκοπεύει να περικόψει 1.200 θέσεις εργασ...,Η κυβέρνηση του προέδρουΝτόναλντ Τραμπ σχεδιάζ...,https://img.bbmd.gr/img/1260/max/82/2025/05/02...,AP.
4,https://www.newsbomb.gr/kosmos/story/1646400/a...,Αμερικανορωσικός «πόλεμος» για την Ημέρα της Ν...,2025-05-02,22:38:19+03:00,ΚΟΣΜΟΣ,Newsbomb,Τι ανέφερε η εκπρόσωπος του ρωσικού Υπουργείου...,Τη δική της απάντηση έδωσε η Μόσχα στον Αμερικ...,https://img.bbmd.gr/img/1260/max/82/2025/05/02...,
...,...,...,...,...,...,...,...,...,...,...
2452,https://www.newsbomb.gr/kosmos/story/1509543/t...,"Τραμπ: «Αν η Ευρώπη δεχτεί επίθεση, οι ΗΠΑ δεν...",2024-01-11,08:09:22+02:00,ΚΟΣΜΟΣ,Newsbomb,Ο Ντόναλντ Τραμπ είχε πει το 2020 στους Ευρωπα...,Ο Αμερικανόςπρώην πρόεδρος Ντόναλντ Τραμπείχε ...,https://img.bbmd.gr/img/1260/max/82/2024/01/11...,O Nτόναλντ Τραμπ\nΑΡ
2453,https://www.newsbomb.gr/kosmos/story/1509454/d...,Ντόναλτ Τραμπ: Μεγάλο προβάδισμα για το χρίσμα...,2024-01-10,20:33:56+02:00,ΚΟΣΜΟΣ,Newsbomb,Ο πρώην πρόεδρος των ΗΠΑ είναι ο προτιμώμενος ...,Ενόψει των προεδρικών εκλογών στις ΗΠΑ μόλις π...,https://img.bbmd.gr/img/1260/max/82/2024/01/10...,Photo: AP
2454,https://www.newsbomb.gr/kosmos/story/1509430/i...,ΗΠΑ: Απεβίωσε η μητέρα της Μελάνια Τραμπ σε ηλ...,2024-01-10,18:59:02+02:00,ΚΟΣΜΟΣ,Newsbomb,"Η Αμαλία Κναβς, σλοβενικής υπηκοότητας, εγκατα...","Σε ηλικία 78 ετών απεβίωσε ηΑμαλία Κναβς, η μη...",https://img.bbmd.gr/img/1260/max/82/2024/01/10...,H μητέρα της Μελάνια Τραμπ Amalija Knavs\nPhot...
2455,https://www.newsbomb.gr/kosmos/story/1509191/m...,Μελάνια Τραμπ: Πέθανε η μητέρα της σε ηλικία 7...,2024-01-10,07:35:33+02:00,ΚΟΣΜΟΣ,Newsbomb,"Η Aμαλία Κναβς, μητέρα της Mελάνια Τραμπ, πέθα...",«Με βαθιά λύπη ανακοινώνω τον θάνατο της αγαπη...,https://img.bbmd.gr/img/1260/max/82/2024/01/10...,Πέθανε στα 78 της η πεθερά του Ντόναλντ Τραμπ...


In [ ]:
# prompt: βρες τα άρρθα από τις 20/11/2024 μέχρι τον απριλιο του 25

from datetime import datetime

# Convert the 'date' column to datetime objects
articles_df['date'] = pd.to_datetime(articles_df['date'], errors='coerce')

# Define the start and end dates
start_date = datetime(2024, 11, 20)
end_date = datetime(2025, 4, 30) # Assuming 'Απρίλιο του 25' means end of April 2025

# Filter the DataFrame
filtered_articles_df = articles_df[(articles_df['date'] >= start_date) & (articles_df['date'] <= end_date)].copy()

# Display the filtered DataFrame
print(f"\nFound {len(filtered_articles_df)} articles between {start_date.date()} and {end_date.date()}.")
filtered_articles_df


Found 1473 articles between 2024-11-20 and 2025-04-30.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
29,https://www.newsbomb.gr/kosmos/story/1645898/s...,«Σχέδιο Β» της Ευρώπης εάν ο Τραμπ εγκαταλείψε...,2025-04-30,22:43:56+03:00,ΚΟΣΜΟΣ,Newsbomb,"«Υπάρχουν εργαλεία στα χέρια των Αμερικανών, π...",ΗΕυρωπαϊκή Ένωσηετοιμάζει «σχέδιο Β» για τιςοι...,https://img.bbmd.gr/img/1260/max/82/2025/04/30...,AP.
30,https://www.newsbomb.gr/kosmos/story/1645874/t...,Τραμπ: Είμαστε έτοιμοι να υπογράψουμε και σήμε...,2025-04-30,21:06:03+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο πρόεδρος των Ηνωμένων Πολιτειών Ντόναλντ Τρα...,"Ο Ντόναλντ Τραμπ, απαντώντας σε ερώτηση δημοσι...",https://img.bbmd.gr/img/1260/max/82/2025/04/30...,Ο πρόεδρος Ντόναλντ Τραμπ μιλάει κατά τη διάρκ...
31,https://www.newsbomb.gr/kosmos/story/1645864/t...,Τραμπ: «Η Κίνα τα πάει πολύ άσχημα αυτή τη στι...,2025-04-30,20:18:56+03:00,ΚΟΣΜΟΣ,Newsbomb,Έκθεση δείχνει συρρίκνωση της κινεζικής εργοστ...,Τη διαπίστωση ότι η οικονομία τηςΚίνας«πηγαίνε...,https://img.bbmd.gr/img/1260/max/82/2025/04/30...,Ο πρόεδρος Ντόναλντ Τραμπ μιλάει κατά τη διάρκ...
32,https://www.newsbomb.gr/kosmos/story/1645860/i...,ΗΠΑ: Κινδυνεύει με απέλαση η Αμάλ Κλούνεΐ - Ο ...,2025-04-30,20:05:44+03:00,ΚΟΣΜΟΣ,Newsbomb,Πιθανή απαγόρευση εισόδου στις ΗΠΑ για τη γνωσ...,"Η Αμάλ Κλούνεΐ,δικηγόρος ανθρωπίνων δικαιωμάτω...",https://img.bbmd.gr/img/1260/max/82/2025/04/30...,Ο George Clooney και η σύζυγός του Amal Cloone...
33,https://www.newsbomb.gr/kosmos/story/1645828/s...,Συμφωνία ΗΠΑ-Ουκρανίας: Εμπόδια της τελευταίας...,2025-04-30,17:52:52+03:00,ΚΟΣΜΟΣ,Newsbomb,Αιφνίδιες διαφωνίες καθυστερούν την υπογραφή κ...,Ησυμφωνία Ηνωμένων Πολιτειών και Ουκρανίας για...,https://img.bbmd.gr/img/1260/max/82/2025/04/26...,AP Photo/Ben Curtis
...,...,...,...,...,...,...,...,...,...,...
1497,https://www.newsbomb.gr/kosmos/story/1603583/m...,Μαρκ Ρόουαν: Φαβορί για υπουργός Οικονομικών τ...,2024-11-20,10:27:19+02:00,ΚΟΣΜΟΣ,Newsbomb,Ο Mαρκ Ρόουαν αναδεικνύεται ως ο κορυφαίος υπο...,Ο δισεκατομμυριούχος ιδιωτικών επενδυτικών κεφ...,https://img.bbmd.gr/img/1260/max/82/2024/11/20...,
1498,https://www.newsbomb.gr/kosmos/story/1603570/h...,HΠΑ: Τι είναι ο ξέφρενος «χορός Τραμπ» και για...,2024-11-20,09:36:56+02:00,ΚΟΣΜΟΣ,Newsbomb,Ένας χορός που διαδόθηκε από τον Ντόναλντ Τραμ...,Ένας χορός που διαδόθηκε από τονεκλεγμένο πρόε...,https://img.bbmd.gr/img/1260/max/82/2024/11/20...,AP
1499,https://www.newsbomb.gr/kosmos/story/1603577/t...,Τουρκία προς Τραμπ: «Παράθυρο» για περιορισμέν...,2024-11-20,09:28:36+02:00,ΚΟΣΜΟΣ,Newsbomb,Η επιδίωξη της Άγκυρας αναφορικά με το εξοπλισ...,ΤοNewsbomb.grείχε γράψει από τον Αύγουστο για ...,https://img.bbmd.gr/img/1260/max/82/2024/11/20...,
1500,https://www.newsbomb.gr/kosmos/story/1603561/r...,Πούτιν: «Έτοιμος» να συζητήσει με τον Τραμπ κα...,2024-11-20,08:46:56+02:00,ΚΟΣΜΟΣ,Newsbomb,Ο Βλαντιμίρ Πούτιν είναι ανοιχτός στο ενδεχόμε...,Ο Βλαντιμίρ Πούτιν είναι ανοιχτός να συζητήσει...,https://img.bbmd.gr/img/1260/max/82/2024/11/20...,Associated Press


In [ ]:
duplicates_url = filtered_articles_df[filtered_articles_df.duplicated(subset=['article_url'], keep=False)]
print(f"\nArticles with duplicate URLs: {len(duplicates_url)}")
if not duplicates_url.empty:
    print(duplicates_url[['article_url', 'title']].sort_values(by='article_url'))

# Check for articles without title
no_title = filtered_articles_df[filtered_articles_df['title'].isnull() | (filtered_articles_df['title'].str.strip() == '')]
print(f"\nArticles without a title: {len(no_title)}")
if not no_title.empty:
    print(no_title[['article_url', 'title']])

# Check for articles without text body
no_text = filtered_articles_df[filtered_articles_df['text_body'].isnull() | (filtered_articles_df['text_body'].str.strip() == '')]
print(f"\nArticles without text body: {len(no_text)}")
if not no_text.empty:
    print(no_text[['article_url', 'title']])


Articles with duplicate URLs: 0

Articles without a title: 0

Articles without text body: 3
                                            article_url  \
1089  https://www.newsbomb.gr/kosmos/video/1622239/d...   
1220  https://www.newsbomb.gr/kosmos/video/1620247/g...   
1240  https://www.newsbomb.gr/kosmos/video/1620045/d...   

                                                  title  
1089  Ντόναλντ Τραμπ: Τι περιείχαν τα gift bags που ...  
1220  Γερουσιαστής «έσκασε μύτη» στην ορκωμοσία Τραμ...  
1240  Ντόναλντ Τραμπ: Χόρεψε ξανά συνοδεία του συγκρ...  


In [ ]:
trump_articles_25 = filtered_articles_df

In [ ]:
trump_articles_25 = trump_articles_25.drop([1089,1220,1240])

In [ ]:
from google.colab import drive

In [ ]:
scrape_newsbomb_articles_pages(trump_teasers_df).to_csv('/content/drive/MyDrive/trump_articles_df_2025.csv', index=False)

Processing rows: 100%|██████████| 2457/2457 [1:46:46<00:00,  2.61s/it]



Scraping completed. Fetshed 2457 articles.
